Useful links

1. For the architecture https://towardsdatascience.com/deep-learning-for-specific-information-extraction-from-unstructured-texts-12c5b9dceada
2. https://androidkt.com/multi-label-text-classification-in-tensorflow-keras/
3. https://keras.io/preprocessing/sequence/
4. https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/ ( Not really)
5. For deep learning using word embeddings https://stackabuse.com/python-for-nlp-multi-label-text-classification-with-keras/



In [2]:
import spacy
import pandas as pd
from tqdm import tqdm

In [3]:
DATA_DIR = "../../data/processed/"
INPUT_FILE_NAME = 'final_squash15_with_pos_ner_tm.parquet'

In [4]:
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags,squash15_tags,pos_sequence,ner_sequence,tm
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"b'[""thank"", ""chris"", ""truly"", ""great"", ""honor""...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie...","culture,politics,science,global issues,technology",VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,"[0.04325945698517057, 0.0, 0.00142482934694180..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"b'[""term"", ""invention"", ""like"", ""tell"", ""tale""...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system...","design,global issues",NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,"[0.013287880838036227, 0.0, 0.0, 0.00511725094..."
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"b'[""public"", ""dewey"", ""long"", ""ago"", ""observe""...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c...","science,culture,politics,global issues,business",ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,"[0.0, 0.006699599134802422, 0.0, 0.00564851883..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"b'[""want"", ""start"", ""say"", ""houston"", ""problem...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren...","design,science,business",VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,"[0.040282108339079505, 0.03732895646484358, 0...."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"b'[""want"", ""talk"", ""background"", ""idea"", ""car""...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog...","design,technology,business,science",VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,"[0.08049208168957463, 0.0, 0.0, 0.008031187136..."


In [5]:
df.iloc[:,:14].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2313 entries, 0 to 2312
Data columns (total 14 columns):
speaker                    2313 non-null object
headline                   2313 non-null object
description                2313 non-null object
duration                   2313 non-null object
tags                       2313 non-null object
transcript                 2313 non-null object
WC                         2313 non-null float64
clean_transcript           2313 non-null object
clean_transcript_string    2313 non-null object
sim_tags                   2313 non-null object
squash15_tags              2313 non-null object
pos_sequence               2313 non-null object
ner_sequence               2313 non-null object
tm                         2313 non-null object
dtypes: float64(1), object(13)
memory usage: 253.1+ KB


In [6]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
def compute_tag_ratio(target_column, df=df):
    tags = df[target_column].str.replace(', ',',').str.lower().str.strip()
    split_tags = tags.str.split(',')
    tag_counts_per_talk = split_tags.apply(len)

    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags = pd.Series(joined_tags)

    tag_counts = all_tags.value_counts().rename_axis(target_column).reset_index(name='counts')
    tag_counts['no_count'] = len(df)-tag_counts['counts']
    tag_counts['ratio'] = tag_counts['counts']/tag_counts['no_count']
    tag_counts['overall_ratio'] = tag_counts['counts']/(tag_counts['no_count'] + tag_counts['counts'])
    return tag_counts

#print(compute_tag_ratio('squash3_tags', df))
squashed_tag_counts = compute_tag_ratio('squash15_tags', df)
print_full_dataframe(squashed_tag_counts)


    squash15_tags  counts  no_count     ratio  overall_ratio
0         science    1467       846  1.734043       0.634241
1         culture    1155      1158  0.997409       0.499351
2      technology     787      1526  0.515727       0.340251
3   global issues     679      1634  0.415545       0.293558
4          design     477      1836  0.259804       0.206226
5         history     385      1928  0.199689       0.166450
6        business     349      1964  0.177699       0.150886
7   entertainment     285      2028  0.140533       0.123217
8           media     279      2034  0.137168       0.120623
9    biomechanics     220      2093  0.105112       0.095115
10   biodiversity     218      2095  0.104057       0.094250
11         future     218      2095  0.104057       0.094250
12       humanity     217      2096  0.103531       0.093818
13       politics     199      2114  0.094134       0.086035
14  communication     185      2128  0.086936       0.079983


In [7]:
print(squashed_tag_counts['overall_ratio'][0])

0.6342412451361867


# 3. Feature Extraction via Deep learning

## 3.1 Create one hot encoding

In [8]:
joined_tags = df['squash15_tags'].str.cat(sep=',').split(',')
all_tags = pd.Series(joined_tags).str.strip().str.lower()
all_tags = list(dict.fromkeys(all_tags))
try:
    all_tags.remove('')
except:
    pass
print(all_tags)
print(len(all_tags))

['culture', 'politics', 'science', 'global issues', 'technology', 'design', 'business', 'biomechanics', 'biodiversity', 'media', 'entertainment', 'history', 'future', 'communication', 'humanity']
15


In [9]:
def create_one_hot_encode(df=df):
    complete_transcripts_tags = []
    for rows, value in df.iterrows():
        one_hot_encoding = [0] * len(all_tags)
        headline = [value['headline']]
        transcript = [value['clean_transcript_string']]
        pos_sequence = [value['pos_sequence']]
        ner_sequence = [value['ner_sequence']]
        indiv_tags = value['squash15_tags'].split(',')
        for tags in indiv_tags:
            if tags == '':
                continue
            index = all_tags.index(tags.lower().lstrip(' '))
            one_hot_encoding[index] = 1
        indiv_transcript_tags = headline + transcript + pos_sequence + ner_sequence + one_hot_encoding
        complete_transcripts_tags.append(indiv_transcript_tags)
    return pd.DataFrame(complete_transcripts_tags, columns=['headline', 'transcript', 'pos_sequence', 'ner_sequence'] + all_tags)

In [10]:
df = create_one_hot_encode()
df

,headline,transcript,pos_sequence,ner_sequence,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
0,Averting the climate crisis,thank chris truly great honor opportunity come...,VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,Simple designs to save a life,term invention like tell tale favorite project...,NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,How to rebuild a broken state,public dewey long ago observe constitute discu...,ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0
3,The real future of space exploration,want start say houston problem enter second ge...,VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
4,Great cars are great art,want talk background idea car art actually mea...,VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2308,Why glass towers are bad for city life -- and ...,imagine walk even discover everybody room look...,VERB NOUN ADV VERB PRON NOUN VERB ADV NOUN NOU...,ORG GPE ORG GPE GPE GPE GPE GPE GPE GPE GPE PE...,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2309,What happens in your brain when you pay attent...,pay close attention easy attention pull differ...,VERB ADJ NOUN ADJ NOUN VERB ADJ NOUN NOUN NOUN...,ORDINAL PERSON PRODUCT DATE DATE,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2310,Why you should define your fears instead of yo...,happy pic take senior college right dance prac...,ADJ PROPN VERB ADJ NOUN ADJ NOUN NOUN ADJ VERB...,DATE PERSON ORG PERSON PERSON GPE PERSON GPE O...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2311,12 truths I learned from life and writing,sevenyearold grandson sleep hall wake lot morn...,PROPN PROPN PROPN PROPN VERB NOUN NOUN VERB VE...,PERSON PERSON PERSON PERSON PERSON DATE CARDIN...,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1


In [11]:
def get_target_column(target_tag, df=df):
    '''Returns a dataframe of a single tag
    headline | transcript | pos_sequence | ner_sequence | <tag>
    '''
    return df[['headline', 'transcript','pos_sequence', 'ner_sequence', target_tag]]
# single_class = get_target_column('culture', df) # Retrieve a single data frame with teh foll

In [12]:
df_x = df[['headline', 'transcript','pos_sequence', 'ner_sequence']] # List of sentences
df_y = df[all_tags] # Dataframe of all the y columns

## 3.2 Perform train test split

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, train_y, valid_y = train_test_split(df_x, df_y, random_state=1000)

In [14]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Conv1D, MaxPooling1D, concatenate
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras import optimizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np

Using TensorFlow backend.


## 3.3 Use word embeddings for the main transcript

In [15]:
# Extract train and test transcripts to list 
X_train_transcripts = X_train['transcript'].tolist()
X_test_transcripts = X_test['transcript'].tolist()
# Extract headline - we will use tfidf because headlines are short 
X_train_headline = X_train['headline'].tolist()
X_test_headline = X_test['headline'].tolist()
# Extract POS tags
X_train_pos_seq= X_train['pos_sequence'].tolist()
X_test_pos_seq = X_test['pos_sequence'].tolist()
# Extract NER tags
X_train_ner_seq = X_train['ner_sequence'].tolist()
X_test_ner_seq = X_test['ner_sequence'].tolist()

In [16]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_transcripts)

X_train_transcripts = tokenizer.texts_to_sequences(X_train_transcripts)
X_test_transcripts = tokenizer.texts_to_sequences(X_test_transcripts)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 500 # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_transcripts = pad_sequences(X_train_transcripts, padding='post', maxlen=maxlen)
X_test_transcripts = pad_sequences(X_test_transcripts, padding='post', maxlen=maxlen)

In [17]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('./glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [18]:
#print(X_train[1])
print(X_train_transcripts.shape)

(1734, 500)


In [19]:
print(train_y['culture'])
print(train_y['culture'].value_counts())
print(train_y['culture'].value_counts()[0])
print(train_y['culture'].value_counts()[1])
print(type(train_y['culture']))

1573    1
236     1
2079    1
1830    0
584     1
       ..
769     1
1372    1
2119    1
599     1
1459    1
Name: culture, Length: 1734, dtype: int64
0    879
1    855
Name: culture, dtype: int64
879
855
<class 'pandas.core.series.Series'>


# Word embeddings on the headline too

In [20]:
# Try word embeddings on the vector 
tokenizer2 = Tokenizer(num_words=500)
tokenizer2.fit_on_texts(X_train_headline)

X_train_headline = tokenizer2.texts_to_sequences(X_train_headline)
X_test_headline = tokenizer2.texts_to_sequences(X_test_headline)

vocab_size2 = len(tokenizer2.word_index) + 1

maxlen2 = 100 # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_headline = pad_sequences(X_train_headline, padding='post', maxlen=maxlen2)
X_test_headline = pad_sequences(X_test_headline, padding='post', maxlen=maxlen2)

# Model

In [21]:
#from keras import backend as K
def compute_class_weight(tag):
    if train_y[tag].value_counts()[0] >train_y[tag].value_counts()[1]:
        class_weight = {0: 1,
                        1: 1/(train_y[tag].value_counts()[1]/train_y[tag].value_counts()[0]),
                       }
    else:
        class_weight = {0: 1/(train_y[tag].value_counts()[0]/train_y[tag].value_counts()[1]),
                    1: 1,
                   }
    return class_weight
print(compute_class_weight('humanity'))

{0: 1, 1: 10.187096774193549}


In [22]:
from keras.utils import plot_model
# define two sets of inputs
inputA = Input(shape=(maxlen2,))
inputB = Input(shape=(maxlen,))
 
# the first branch operates on the first input which is the headline
embedding_layer_hedline = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputA) 
#model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
#model.add(layers.Conv1D(128, 5, activation='relu'))
x = Conv1D(128, 5, activation='relu')(embedding_layer_hedline)
# model.add(layers.GlobalMaxPooling1D())
x = GlobalMaxPooling1D()(x)
x = Dense(10, activation='relu')(x)
X = Dropout(0.2)(x)
x = Dense(4, activation="relu")(x)
# model.add(layers.Dense(10, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# x = Dense(50, activation="relu")(inputA)
# x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
 
# the second branch opreates on the second input
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputB)
y = LSTM(128)(embedding_layer)
y = Dropout(0.2)(y)
y = Dense(4, activation='relu')(y)
y = Model(inputs=inputB, outputs=y)
 
# combine the output of the two branches
combined = concatenate([x.output, y.output])
 
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="sigmoid")(z)
 
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)
print(model.summary())
adam = optimizers.adam(lr=0.0001)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     4228700     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 96, 128)      64128       embedding_1[0][0] 

In [23]:
#predictions = model.predict(X_test)

In [24]:
def get_tag(threshold, predictions):
    ''' Flush predictions to either 1 or 0
    Returns
    [[1], [0], ..., [1]]
    '''
    return [[1 if j > threshold else 0 for j in i.tolist()] for i in predictions]

def get_tag_flat(threshold, predictions):
    ''' Flush predictions to either 1 or 0
    Returns
    [[1], [0], ..., [1]]
    '''
    return [1 if j > threshold else 0 for i in predictions for j in i]
#predictions_flushed = get_tag(0.4)


In [25]:
def compute_tp_tn_fp_fn(y_test, y_pred, classes):
    '''
    parameters
    y_test (list): [1,...,0]
    y_pred (list): [[1], [0], ..., [1]]
    classes: [tag1, tag2, ..., tagn]
    
    Return:
    pre_score = {
        'tag_1': {
            'index': ,
            'tp': ,
            'tn': ,
            'fp': ,
            'fn': 
        }
    }
    '''
    # Create dictionary of tags 
    pre_score = {}
    for index_tag, tag in enumerate(classes):
        pre_score[tag] = {
            'index':index_tag,
            'tp': 0,
            'tn': 0,
            'fp': 0,
            'fn': 0
        }
    for transcript_index, transcript_value in enumerate(y_test):
        if transcript_value == y_pred[transcript_index][0] and transcript_value == 1:
            pre_score[classes[0]]['tp'] += 1
        elif transcript_value == y_pred[transcript_index][0] and transcript_value == 0:
            pre_score[classes[0]]['tn'] += 1
        elif transcript_value != y_pred[transcript_index][0] and transcript_value == 1:
            pre_score[classes[0]]['fn'] += 1
        elif transcript_value != y_pred[transcript_index][0] and transcript_value == 0:
            pre_score[classes[0]]['fp'] += 1
    return pre_score
#scores_preprocess = compute_tp_tn_fp_fn(valid_y, predictions_flushed, ['culture'])

In [26]:
def compute_precision_recall_f1(preprocessed_scores):
    '''
    parameters
    preprocessed_scores = {
        'tag_1': {
            'index': ,
            'tp': ,
            'tn': ,
            'fp': ,
            'fn': 
        }
    }
    return
    preprocessed_scores = {
        'tag_1': {
            'index': ,
            'tp': ,
            'tn': ,
            'fp': ,
            'fn': ,
            'precision': ,
            'recall': ,
            'f1': ,
            'accuracy': ,
        }
    }
    '''
    for key, value in preprocessed_scores.items():
        try:
            precision = value['tp']/(value['tp']+value['fp'])
        except:
            #print('precision issue: {}'.format(key))
            precision = 0.0
        try:
            recall = value['tp']/(value['tp']+value['fn'])
        except:
            #print('recall issue: {}'.format(key))
            recall = 0.0
        try:
            f1 = (2 * precision * recall)/(precision + recall)
        except:
            #print('f1 issue: {}'.format(key))
            f1=0.0
        try:
            accuracy = (value['tp'] + value['tn'])/(value['tp']+value['fn'] + value['fp'] + value['tn'])
        except:
            accuracy = 0.0
        preprocessed_scores[key]['precision'] = round(precision,4)
        preprocessed_scores[key]['recall'] = round(recall,4)
        preprocessed_scores[key]['f1'] = round(f1,4)
        preprocessed_scores[key]['accuracy'] = round(accuracy,4)
    return preprocessed_scores
# final_scores = compute_precision_recall_f1(scores_preprocess)
# print(final_scores)

In [27]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [28]:
def format_scores_df(tag_classes, final_scores):
    '''
    Given the list of classes and the dicitionary of all the classes and their scores, we write to a dataframe
    '''
    precision = []
    recall = []
    f1 = []
    accuracy = []
    for index, value in enumerate(tag_classes):
        precision.append(final_scores[value]['precision'])
        recall.append(final_scores[value]['recall'])
        f1.append(final_scores[value]['f1'])
        accuracy.append(final_scores[value]['accuracy'])
    df_result = pd.DataFrame(list(zip(tag_classes, precision, recall, f1, accuracy)), 
               columns =['class', 'precision', 'recall', 'f1', 'accuracy']) 
    return df_result
# df_results = format_scores_df(['culture'], final_scores)
# print_full_dataframe(df_results)

In [29]:
def get_threshold(tag,valid_y, predictions):
    '''
    tag (string): Specific tag
    valid_y (List of tags): [0,1,1, ... ,1]
    predictions (list of lists)
    '''
    # We want to find the threshold that gives the highest recall and accuracy
    highest_f1 = 0
    f1_i = []
    highest_accuracy_f1 = 0
    accuracy_f1_i = []
    highest_accuracy = 0
    accuracy_i = []
    f1_metrics = [0, 0, 0, 0] # tp, tn, fp, fn
    accuracy_metrics = [0, 0, 0, 0]
    for i in range(0, 100):
        i = i/100
        predictions_flushed = get_tag(i,predictions)
        scores_preprocess = compute_tp_tn_fp_fn(valid_y, predictions_flushed, [tag])
        final_scores = compute_precision_recall_f1(scores_preprocess)
    #     print(final_scores)
        df_results = format_scores_df([tag], final_scores)
        #print(df_results)
        f1 = final_scores[tag]['f1']
        accuracy = df_results.accuracy[0]

        if f1 > highest_f1:
            highest_f1 = f1
            f1_i = [i]
            if accuracy > highest_accuracy_f1:
                highest_accuracy_f1 = accuracy
                accuracy_f1_i = [i]
                f1_metrics[0] = scores_preprocess[tag]['tp']
                f1_metrics[1] = scores_preprocess[tag]['tn']
                f1_metrics[2] = scores_preprocess[tag]['fp']
                f1_metrics[3] = scores_preprocess[tag]['fn']
            elif accuracy == highest_accuracy_f1:
                accuracy_f1_i.append(i)
        elif f1 == highest_f1:
            f1_i.append(i)
            if accuracy > highest_accuracy_f1:
                highest_accuracy_f1 = accuracy
                accuracy_f1_i = [i]
                
            elif accuracy == highest_accuracy_f1:
                accuracy_f1_i.append(i)

        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            accuracy_i = [i]
            accuracy_metrics[0] = scores_preprocess[tag]['tp']
            accuracy_metrics[1] = scores_preprocess[tag]['tn']
            accuracy_metrics[2] = scores_preprocess[tag]['fp']
            accuracy_metrics[3] = scores_preprocess[tag]['fn']
        elif accuracy == highest_accuracy:
            accuracy_i.append(i)

    #     print('\n')

#     print(highest_f1,f1_i)
#     print(highest_accuracy_f1,accuracy_f1_i)
#     print(highest_accuracy,accuracy_i)
    return highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i, f1_metrics, accuracy_metrics

In [ ]:
# tag_results = {}
# for i in range(len(all_tags)):
#     tag = all_tags[i]
#     print(tag)
#     single_class = get_target_column(tag, df)
#     train_y_tag = train_y[tag]
#     valid_y_tag = valid_y[tag]
#     history = model.fit(X_train, train_y_tag, batch_size=32, epochs=4, verbose=1, validation_split=0.2)
#     predictions = model.predict(X_test)
#     highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i = get_threshold(tag,valid_y_tag,predictions)
#     print(highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i)
#     tag_results[tag] = [highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i]
all_tags = ['entertainment', 'history', 'future', 'communication', 'humanity']
tag_results = {}
for i in range(len(all_tags)):
    tag = all_tags[i]
    print(tag)
    train_y_tag = train_y[tag]
    valid_y_tag = valid_y[tag]
    class_weight = compute_class_weight(tag)
    history = model.fit([X_train_headline, X_train_transcripts], train_y_tag, batch_size=32, epochs=4, verbose=1, validation_split=0.2, class_weight=class_weight)
    predictions = model.predict([X_test_headline, X_test_transcripts])
    model.save('{}_transcript_only.h5'.format(tag))
    highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i, f1_metrics, accuracy_metrics = get_threshold(tag,valid_y_tag,predictions)
    print(highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i, f1_metrics, accuracy_metrics)
    tag_results[tag] = [highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i, f1_metrics, accuracy_metrics]

entertainment


Train on 1387 samples, validate on 347 samples
Epoch 1/4
 160/1387 [==>...........................] - ETA: 1:54:30 - loss: 1.2520 - acc: 0.2188

In [ ]:
highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i, f1_metrics, accuracy_metrics = get_threshold(tag,valid_y_tag,predictions)
print(highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i, f1_metrics, accuracy_metrics )

In [ ]:
print(tag_results)

In [ ]:
pd.DataFrame.from_dict(tag_results, orient='index', columns=['highest_f1', 'thresholds_for_highest_f1', 'highest_accuracy_at_highest_f1', 'thresholds_for_highest_accuracy_f1','highest_accuracy','threshold_for_highest_accuracy_i', 'highest_f1_confusion_metrics', 'highest_accuracy_confusion_metrics'])